# Data Cleaning

## 1. Import modules

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import datetime as dt
from plotnine import * # Implementation of ggplot

## 2. Load data

- King County Housing Data.
- House Sales from King-County, WA, USA from May 2014 to May 2015.

In [ ]:
df_hs = pd.read_csv("./data/king_county_houses_sales.csv")

## 3. Inspect the data
1. Meaningful column names in standard forms (lower-case, no whitespaces?)?
2. Right data types for all columns?
3. correct SI-Units?
4. Duplicates? 

#### 3.1 Column Names

In [ ]:
### 1. Show the column names
df_hs.columns

### Create a dictionary for df.rename() 
### Shorten a few columns, define better, more easily understandable names and change area to m^2
new_names = {'id':'id_house', 
            'bedrooms':'n_bedr', 
            'bathrooms': 'n_bathr', 
            'sqft_living': 'a_living_sqm', 
            'sqft_lot': 'a_lot_sqm',
            'floors': 'n_floors',
            'sqft_above': 'a_above_sqm',
            'sqft_basement': 'a_basem_sqm', 
            'view': 'inspected', 
            'sqft_living15': 'a_living_15_sqm', 
            'sqft_lot15': 'a_lot_15_sqm',
            'date': 'date_sold'}

### Apply rename with the dictionary variable for changing column names
df_hs.rename(new_names, axis = 1, inplace = True)
df_hs.columns

### Clear workspace of new_names
del new_names


#### 3.2 Check the data-types

In [ ]:
### 2. Right Data types?
df_hs.shape

#   date_sold to be changed into date
df_hs.date_sold = pd.to_datetime(df_hs.date_sold)
#   yr_renovated numeric but should be as integer or date
df_hs.yr_renovated.sort_values().unique() # either 0, 19340 >= <= 20150 always with last digit 0 or nan
#   Thus divide by 10 to convert to years and to int
df_hs.yr_renovated = df_hs.yr_renovated / 10



##### Clean yr_renovated

In [ ]:


### Function to differentiate btween 0, NaN and valid year values
df_hs.yr_renovated.sort_values().unique()
def group_year(x): 
    if pd.isna(x):
        return "NaN"
    elif x == 0:
        return "0"
    else:
        return "Valid year"

### Copy the dataframe
df_hs_2 = df_hs

### Create a column which defines wether yr_renovated has a NaN value, a 0 value or a valid year
df_hs_2["val_yr_renov"] = df_hs.yr_renovated.apply(group_year)

### Plot a histogram for each of the class to determine, wether 0 and NaN depend on the year the house was built
fig = px.histogram(df_hs_2, x = "yr_built", 
                    facet_row = "val_yr_renov",
                    color = "val_yr_renov")
fig.update_yaxes(matches=None)
fig.show()

del df_hs_2
del group_year

### Assign a NaN to yr_renovated where it is 0
df_hs.yr_renovated = df_hs.yr_renovated.replace(0, np.nan, inplace = False)

### As String - creates 'nan' instead of nan
df_hs.yr_renovated = df_hs.yr_renovated.astype("str").str.replace(".0", "", regex = False)

### cast 'nan' back to nan
df_hs.yr_renovated = df_hs.yr_renovated.replace("nan", np.nan)

### cast back to float. Int does not include the NaN
df_hs.yr_renovated = df_hs.yr_renovated.astype("float")

- Only valid years in yr_renovated show a meaningful pattern - most houses built after 1980 have not been renovated since
- while 0 and NaN values follow the distribution of the whole dataset show the same pattern, only in different numbers
- If one of these two were to represent renovated houses with the year of renovation unknown, a different pattern could be expected, independent of the overall pattern of houses built

#### Inspect yr_build
- No missing values

In [ ]:
# Inspect yr_build
px.histogram(df_hs, x = "yr_built") # seems to make sense
df_hs.yr_built.unique() # No missing values

### 3.3 Correct SI Units?
- Convert Area columns to m^2

In [ ]:
# Area columns to sqm
ind = df_hs.columns.str.contains("_sqm$", regex = True)
df_hs.loc[:,ind] = df_hs.loc[:,ind].apply(lambda x: x/10.764).round(2)
df_hs

In [ ]:
# Values of area - outliers etc?
df_areas = df_hs.loc[:,ind].melt() ### all area columns melted into long format for easier plotting

p1 = ggplot (df_areas, aes (x = "value")) + \
    geom_freqpoly() + \
    facet_wrap ("variable", ncol = 2, scales = "free") + \
    theme(panel_spacing = 0.3)
ggsave (p1, "./plots/area_histograms.png", dpi = 150)

### Overview of the distributon parameters of areas. Looks reasonable and also interested in lower values
df_hs.loc[:,ind].describe()

del df_areas
del ind

- Data of all area columns looks reasonable

### 3.4 Duplicates?
- For duplicated sells: exclude earlier sell of house

In [ ]:
### For duplicated sells: exclude earlier sell of house
### With drop duplicates after sort_values by sale date
df_hs_2 = df_hs.sort_values(by = "date_sold", ascending = False)

df_hs_2.drop_duplicates(subset = "id_house", keep = "first", inplace = True)
df_hs_2.reset_index(inplace = True)

In [ ]:
### Save the cleaned data frame for later use
df_hs_2.to_pickle("./data/df_houses_cleaned.pkl")

In [ ]:
df_hs_2